In [ ]:
import requests
import pandas as pd
import time
import os
from io import StringIO

def truncate(sio):
    sio.truncate(0)
    sio.seek(0)
    return sio

class stock:
    def __init__(self,save_file_path):
        if not os.path.exists(save_file_path):
            os.makedirs(save_file_path)
        pass

    def run(self,stocks_list,months_range,save_file_path):
        for stock in stocks_list: #走過每一個股票
            #建立好每個股票的空的df 
            columns = ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
            my_df = pd.DataFrame(columns = columns)
            for m in months_range: #走過每個月分
                url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date='+ m +'&stockNo='+stock
                print('processing:',url)
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
                res = requests.get(url,headers = headers)
                io_csv_info = StringIO(res.text)
                io_csv_info.readline() # 讀取第一行(股票代號資訊,我們不需要)
                io_csv_info.readline() # columns 內容 我們不需要
                while True: #走過每一天
                    line = io_csv_info.readline()
                #     print(line)
                    info_list = line.split('","')
                    info_list[0]  = info_list[0].strip('"')
                    info_list[-1] = info_list[-1].strip('",\r\n')
                #     info_list
        #             print(info_list)
                    if '"說明:"\r\n' in line :
                        break

                    df = pd.DataFrame([info_list], columns = columns) #每一行建立df 
                    my_df = my_df.append(df , ignore_index = True) #把每一行的df 添加到我的my_df
                truncate(io_csv_info) #清除舊有的記憶體空間資料
                time.sleep(7)
            my_df.to_csv(save_file_path + f'/{stock}_from_{start_month}_to_{end_month}.csv' , encoding = 'utf-8')
            print(f'=====================finished {stock} crawl========================================')
            
            
if __name__ == '__main__':   # 以下內容只會在main 的文本檔執行         
    stocks_list = str(input('stocks list (ex:2317,2330,2303)')).split(',')
    start_month = str(input('month start (ex:2020-01):'))
    end_month   = str(input('month end   (ex:2022-02):'))
    save_file_path = str(input('save file path(ex: ./csv_data)'))

    #產出月份的區間
    months_range = [str(date).replace('-','').split(' ')[0] for date in pd.date_range(start=start_month ,end = end_month,freq = 'MS')]

    S=stock(save_file_path) #初始化 會去確認資料夾
    S.run(stocks_list,months_range,save_file_path) #爬取股票